
# Zillow Rental Affordability ETL
This notebook processes Zillow rental affordability data, including cleaning, restructuring, and handling missing values.

## 1. Import Libraries
We begin by importing the necessary Python libraries.

In [330]:
import pandas as pd

## 2. Load the Data
We read the Zillow rental affordability dataset from a CSV file.

In [ ]:
df_zillow = pd.read_csv(
    r'C:\Users\isabe\PycharmProjects\GSB 520 ETL\Data\Metro_new_renter_affordability_uc_sfrcondomfr_sm_sa_month.csv')

## 3. Preview the Data
We display the first few rows of the dataset to understand its structure.

In [332]:
df_zillow.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,...,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31
0,102001,0,United States,country,NaN,0.260315,0.260700,0.261297,0.261654,0.261915,...,0.290183,0.289985,0.289987,0.289937,0.289916,0.289881,0.289899,0.290068,0.290065,0.290958
1,394913,1,"New York, NY",msa,NY,0.376445,0.378414,0.379987,0.381039,0.381047,...,0.377414,0.377039,0.376682,0.376297,0.375988,0.375566,0.376005,0.377097,0.378269,0.380136
2,753899,2,"Los Angeles, CA",msa,CA,0.350640,0.351405,0.352736,0.353743,0.354311,...,0.362304,0.362099,0.362117,0.362203,0.362026,0.362168,0.362401,0.363089,0.363308,0.365414
3,394463,3,"Chicago, IL",msa,IL,0.258110,0.257762,0.257956,0.258126,0.258385,...,0.260503,0.260613,0.260892,0.261152,0.261492,0.261961,0.262598,0.263459,0.263942,0.265141
4,394514,4,"Dallas, TX",msa,TX,0.210304,0.210565,0.211446,0.212933,0.213913,...,0.229983,0.229243,0.228563,0.227807,0.227418,0.227082,0.226815,0.226329,0.225882,0.226002


## 4. Data Cleaning and Restructuring
We remove unnecessary columns, reshape the dataset into long format, and convert date values.

In [ ]:
# Drop any unnamed or empty columns
df_zillow = df_zillow.loc[:, ~df_zillow.columns.str.contains("^Unnamed")]

# Convert from wide to long format
df_long = df_zillow.melt(
    id_vars=["RegionID", "SizeRank", "RegionName", "RegionType", "StateName"],
    var_name="Date",
    value_name="New_Renter_Affordability"
)

# Convert 'Date' column to datetime format
df_long["Date"] = pd.to_datetime(df_long["Date"], errors="coerce")

# Drop rows with missing 'Date' values (if any errors in conversion)
df_long = df_long.dropna(subset=["Date"])

# Optional: Handle missing values in 'New_Renter_Affordability'
df_long["New_Renter_Affordability"] = df_long["New_Renter_Affordability"].astype(float)


## 5. Verify the Transformed Data
We check the structure and contents of the transformed data.

In [334]:
df_long.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,Date,New_Renter_Affordability
0,102001,0,United States,country,NaN,2015-01-31,0.260315
1,394913,1,"New York, NY",msa,NY,2015-01-31,0.376445
2,753899,2,"Los Angeles, CA",msa,CA,2015-01-31,0.350640
3,394463,3,"Chicago, IL",msa,IL,2015-01-31,0.258110
4,394514,4,"Dallas, TX",msa,TX,2015-01-31,0.210304


### Check Data for Washington, D.C.
We filter the dataset for Washington, D.C. to verify the transformation.

In [335]:
df_long[df_long["RegionName"] == "Washington, DC"]

,RegionID,SizeRank,RegionName,RegionType,StateName,Date,New_Renter_Affordability
6,395209,6,"Washington, DC",msa,VA,2015-01-31,0.222851
397,395209,6,"Washington, DC",msa,VA,2015-02-28,0.223118
788,395209,6,"Washington, DC",msa,VA,2015-03-31,0.223520
1179,395209,6,"Washington, DC",msa,VA,2015-04-30,0.223586
1570,395209,6,"Washington, DC",msa,VA,2015-05-31,0.223726
...,...,...,...,...,...,...,...
45362,395209,6,"Washington, DC",msa,VA,2024-09-30,0.218944
45753,395209,6,"Washington, DC",msa,VA,2024-10-31,0.219103
46144,395209,6,"Washington, DC",msa,VA,2024-11-30,0.219447
46535,395209,6,"Washington, DC",msa,VA,2024-12-31,0.219824


### Count of Region Types
We count the different region types present in the dataset.

In [336]:
df_long['RegionType'].value_counts()

RegionType
msa        47190
country      121
Name: count, dtype: int64

## 6. Extract and Standardize City Names
We extract city names from region names and standardize naming conventions.

In [337]:
# Function to fill City column based on RegionType
def extract_city(region_name, region_type):
    if region_type == "msa" and region_name != "Washington, DC":
        return region_name.split(",")[0].strip()
    elif region_name == "Washington, DC":
        return "Washington, D.C."
    return region_name  # Keep original value for non-msa rows

# Apply function to create City column
df_long["City"] = df_long.apply(lambda row: extract_city(row["RegionName"], row["RegionType"]), axis=1)

# Rename 'StateName' to 'State'
df_long = df_long.rename(columns={"StateName": "State"})

# Reorder columns for clarity
df_long = df_long[
    ["RegionID", "SizeRank", "City", "RegionType", "State", "Date", "New_Renter_Affordability"]
]

## 7. Final Data Check
We verify the final structure of the cleaned dataset.

In [338]:
df_long.head()

,RegionID,SizeRank,City,RegionType,State,Date,New_Renter_Affordability
0,102001,0,United States,country,NaN,2015-01-31,0.260315
1,394913,1,New York,msa,NY,2015-01-31,0.376445
2,753899,2,Los Angeles,msa,CA,2015-01-31,0.350640
3,394463,3,Chicago,msa,IL,2015-01-31,0.258110
4,394514,4,Dallas,msa,TX,2015-01-31,0.210304


## 8. Filter for Metropolitan Statistical Areas (MSAs)
We focus on MSA regions for further analysis.

In [339]:
df_zillow_cleaned = df_long[df_long["RegionType"] == "msa"]
df_zillow_cleaned[df_zillow_cleaned['New_Renter_Affordability'].isna()].head()

,RegionID,SizeRank,City,RegionType,State,Date,New_Renter_Affordability
84,395115,85,Springfield,msa,MA,2015-01-31,NaN
89,395143,91,Syracuse,msa,NY,2015-01-31,NaN
96,394711,98,Jackson,msa,MS,2015-01-31,NaN
104,395245,106,Youngstown,msa,OH,2015-01-31,NaN
114,394761,116,Lafayette,msa,LA,2015-01-31,NaN


### Identify Missing Values
We capture the indexes of missing values for later reference.

In [340]:
na_indexes = df_zillow_cleaned[df_zillow_cleaned['New_Renter_Affordability'].isna()].head().index

## 9. Fill Missing Values with State-Level Averages
We handle missing values by filling them with state-level averages.

In [341]:
# Convert 'Date' to datetime
df_zillow_cleaned.loc[:, "Date"] = pd.to_datetime(df_zillow_cleaned["Date"])

# Compute state-level mean affordability by date
state_avg = df_zillow_cleaned.groupby(["State", "Date"])["New_Renter_Affordability"].transform("mean")

# Fill missing values using .loc[:]
df_zillow_cleaned.loc[:, "New_Renter_Affordability"] = df_zillow_cleaned["New_Renter_Affordability"].fillna(state_avg)

### Check Replaced Values
We verify that previously missing values have been filled.

In [342]:
# Display values at former NaN indexes
df_zillow_cleaned.loc[na_indexes, ["City", "State", "Date", "New_Renter_Affordability"]]

,City,State,Date,New_Renter_Affordability
84,Springfield,MA,2015-01-31,0.251393
89,Syracuse,NY,2015-01-31,0.237766
96,Jackson,MS,2015-01-31,0.227809
104,Youngstown,OH,2015-01-31,0.177081
114,Lafayette,LA,2015-01-31,0.260903


## 10. Save the Cleaned Dataset
Finally, we save the cleaned dataset for further analysis.

In [343]:
df_zillow_cleaned.to_csv(
    r"C:\Users\isabe\PycharmProjects\GSB 520 ETL\Clean Data\zillow_cleaned.csv", index=False)